# MONet

## Git Setup

In [20]:
import getpass

# Necessary to push to git
!git remote add origin_pwd https://{input("User: ")}:{getpass.getpass("Auth Token: ")}@github.com/{username}/ia368cc-project.git

Auth Token: ··········


In [8]:
# Necessary to commit
!git config --global user.email "luiza.pozzobon@gmail.com"
!git config --global user.name "Luiza Pozzobon"

In [23]:
!git push origin_pwd luiza/monet_teste

Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 18.42 KiB | 3.68 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/luizapozzobon/ia368cc-project.git
   836d1db..78d91b8  luiza/monet_teste -> luiza/monet_teste


In [24]:
!git status

On branch luiza/monet_teste
Your branch is ahead of 'origin/luiza/monet_teste' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)
  (commit or discard the untracked or modified content in submodules)

	modified:   notebooks/luiza_01_monet-test.ipynb
	modified:   references/MONet-pytorch (modified content)

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	checkpoints/

no changes added to commit (use "git add" and/or "git commit -a")


## Environment and repository setup

In [1]:
import sys
from pathlib import Path
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
PROJECT_DIR = Path("/content/gdrive/MyDrive/2021-IA368CC/project/ia368cc-project/").resolve()
NOTEBOOKS_DIR = PROJECT_DIR / "notebooks"
REFERENCES_DIR = PROJECT_DIR / "references"

# Just to make sure we're in the correct folder
assert PROJECT_DIR.exists()
assert NOTEBOOKS_DIR.exists()
assert REFERENCES_DIR.exists()

%cd {PROJECT_DIR}

/content/gdrive/MyDrive/2021-IA368CC/project/ia368cc-project


In [ ]:
# add references directories to path
sys.path.append(REFERENCES_DIR / "MONet-pytorch")

In [ ]:
def install_requirements():
  """Install project's requirement files."""
  !pip install -q -r {PROJECT_DIR / "requirements.txt"}
  !pip install -q -r {PROJECT_DIR / "extra-requirements.txt"}
  !pip install -q -r {PROJECT_DIR / "monet-requirements.txt"}

In [ ]:
def setup_references():
  """Initialize git submodules.

  Reference: https://git-scm.com/book/en/v2/Git-Tools-Submodules
  
  """
  !cd references/MONet-pytorch
  !git submodule init
  !git submodule update

In [ ]:
install_requirements()

     |████████████████████████████████| 131 kB 5.2 MB/s 
     |████████████████████████████████| 1.6 MB 26.2 MB/s 
     |████████████████████████████████| 78 kB 6.5 MB/s 
     |████████████████████████████████| 62 kB 563 kB/s 
     |████████████████████████████████| 129 kB 30.9 MB/s 
     |████████████████████████████████| 149 kB 45.5 MB/s 
     |████████████████████████████████| 178 kB 46.9 MB/s 
     |████████████████████████████████| 97 kB 6.5 MB/s 
     |████████████████████████████████| 1.9 MB 30.4 MB/s 
     |████████████████████████████████| 73 kB 1.6 MB/s 
     |████████████████████████████████| 1.5 MB 37.2 MB/s 
     |████████████████████████████████| 59 kB 6.1 MB/s 
     |████████████████████████████████| 3.3 MB 36.9 MB/s 
     |████████████████████████████████| 26.9 MB 99 kB/s 
     |████████████████████████████████| 124 kB 31.7 MB/s 
     |████████████████████████████████| 788 kB 37.1 MB/s 
     |████████████████████████████████| 133 kB 47.4 MB/s 
     |████████████████████

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
  Cloning https://github.com/takuseno/d4rl-atari to /tmp/pip-req-build-zqmdrowl
  Running command git clone -q https://github.com/takuseno/d4rl-atari /tmp/pip-req-build-zqmdrowl
     |██████████████▋                 | 834.1 MB 35.5 MB/s eta 0:00:28tcmalloc: large alloc 1147494400 bytes == 0x55eecac8c000 @  0x7fcccd9b3615 0x55ee912be4cc 0x55ee9139e47a 0x55ee912c12ed 0x55ee913b2e1d 0x55ee91334e99 0x55ee9132f9ee 0x55ee912c2bda 0x55ee91334d00 0x55ee9132f9ee 0x55ee912c2bda 0x55ee91331737 0x55ee913b3c66 0x55ee91330daf 0x55ee913b3c66 0x55ee91330daf 0x55ee913b3c66 0x55ee91330daf 0x55ee912c3039 0x55ee91306409 0x55ee912c1c52 0x55ee91334c25 0x55ee9132f9ee 0x55ee912c2bda 0x55ee91331737 0x55ee9132f9ee 0x55ee912c2bda 0x55ee91330915 0x55ee912c2afa 0x55ee91330c0d 0x55ee9132f9ee
     |██████████████████▌             | 1055.7 MB 1.6 MB/s eta 0:07:47tcmalloc: large alloc 1434370048 bytes == 0x55ef0f2e2000 @  0x7fcccd9b3615 0x55ee9

In [ ]:
setup_references()

Submodule 'references/MONet-pytorch' (https://github.com/baudm/MONet-pytorch) registered for path 'references/MONet-pytorch'
Cloning into '/content/gdrive/My Drive/2021-IA368CC/project/ia368cc-project/references/MONet-pytorch'...
Submodule path 'references/MONet-pytorch': checked out '75239c6e74e4947b6728ffb615e8e81b24718a7e'


## MONet Setup

In [ ]:
def setup_monet_clevr():
  """Setup MONet's CLEVR experiment. 
  
  Instructions from the repo: https://github.com/baudm/MONet-pytorch
  
  """
  %cd {REFERENCES_DIR / "MONet-pytorch/datasets"}
  if not Path("CLEVR_v1.0.zip").exists():
    !wget -cN "https://dl.fbaipublicfiles.com/clevr/CLEVR_v1.0.zip"
  !unzip -n -qq CLEVR_v1.0.zip # quiet and never overwrites what's already there
  %cd {PROJECT_DIR}

In [ ]:
setup_monet_clevr()

/content/gdrive/MyDrive/2021-IA368CC/project/ia368cc-project/references/MONet-pytorch/datasets
/content/gdrive/MyDrive/2021-IA368CC/project/ia368cc-project


## Run MONet default training file

### Setup ngrok tunnel so we can access the local web server

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

get_ipython().system_raw('./ngrok http 8097 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
 "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Train

In [ ]:
!python {REFERENCES_DIR / "MONet-pytorch/train.py"} --dataroot {REFERENCES_DIR / "MONet-pytorch/datasets/CLEVR_v1.0"} --name clevr_monet --model monet

----------------- Options ---------------
               batch_size: 64                            
                     beta: 0.5                           
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 192                           
                 dataroot: /content/gdrive/MyDrive/2021-IA368CC/project/ia368cc-project/references/MONet-pytorch/datasets/CLEVR_v1.0	[default: None]
             dataset_mode: clevr                         
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 11                            
             display_port: 8097                          
           display_server: http://localhost            

## Decompose MONet-pytorch so we can apply to other datasets

In [ ]:
# MONet imports
import time
from options.train_options import TrainOptions
from data import create_dataset
from models import create_model
from util.visualizer import Visualizer

In [ ]:
opt = TrainOptions().parse()   # get training options
dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options
dataset_size = len(dataset)    # get the number of images in the dataset.
print('The number of training images = %d' % dataset_size)

model = create_model(opt)      # create a model given opt.model and other options
model.setup(opt)               # regular setup: load and print networks; create schedulers
visualizer = Visualizer(opt)   # create a visualizer that display/save images and plots
total_iters = 0                # the total number of training iterations

for epoch in range(opt.epoch_count, opt.niter + opt.niter_decay + 1):    # outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
    epoch_start_time = time.time()  # timer for entire epoch
    iter_data_time = time.time()    # timer for data loading per iteration
    epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch

    for i, data in enumerate(dataset):  # inner loop within one epoch
        iter_start_time = time.time()  # timer for computation per iteration
        if total_iters % opt.print_freq == 0:
            t_data = iter_start_time - iter_data_time
        visualizer.reset()
        total_iters += opt.batch_size
        epoch_iter += opt.batch_size
        model.set_input(data)         # unpack data from dataset and apply preprocessing
        model.optimize_parameters()   # calculate loss functions, get gradients, update network weights

        if total_iters % opt.display_freq == 0:   # display images on visdom and save images to a HTML file
            save_result = total_iters % opt.update_html_freq == 0
            model.compute_visuals()
            visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

        if total_iters % opt.print_freq == 0:    # print training losses and save logging information to the disk
            losses = model.get_current_losses()
            t_comp = (time.time() - iter_start_time) / opt.batch_size
            visualizer.print_current_losses(epoch, epoch_iter, losses, t_comp, t_data)
            if opt.display_id > 0:
                visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, losses)

        if total_iters % opt.save_latest_freq == 0:   # cache our latest model every <save_latest_freq> iterations
            print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
            save_suffix = 'iter_%d' % total_iters if opt.save_by_iter else 'latest'
            model.save_networks(save_suffix)

        iter_data_time = time.time()
    if epoch % opt.save_epoch_freq == 0:              # cache our model every <save_epoch_freq> epochs
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
        model.save_networks('latest')
        model.save_networks(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' % (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))
    model.update_learning_rate()                     # update learning rates at the end of every epoch.
